In [ ]:
#12pm 19th nov

import numpy as np
import pandas as pd
import torch
from torch import nn
from transformers import BertTokenizer, BertModel
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import classification_report
import torch.nn.functional as F
from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder

# Function to encode labels
def encode_labels(df, target_column, severity_column):
    le_target = LabelEncoder()
    le_severity = LabelEncoder()

    # Encode 'Target' and 'Severity' labels into integers
    df[target_column] = le_target.fit_transform(df[target_column])
    df[severity_column] = le_severity.fit_transform(df[severity_column])

    return df, le_target, le_severity

# Custom Dataset
class TargetSeverityDataset(Dataset):
    def __init__(self, texts, target_labels, severity_labels, tokenizer, max_len=128):
        self.texts = texts
        self.target_labels = target_labels
        self.severity_labels = severity_labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])
        target_label = self.target_labels[idx]
        severity_label = self.severity_labels[idx]

        encoding = self.tokenizer(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )

        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'target_label': torch.tensor(target_label, dtype=torch.long),
            'severity_label': torch.tensor(severity_label, dtype=torch.long)
        }

# Multi-task Model with Softmax
# Updated Multi-task Model
class MultiTaskTargetSeverityClassifier(nn.Module):
    def __init__(self, bert_model, dropout_rate=0.4, num_classes=4):
        super().__init__()
        self.bert = bert_model
        hidden_size = bert_model.config.hidden_size

        # Common layers
        self.dropout = nn.Dropout(dropout_rate)
        self.dense1 = nn.Linear(hidden_size, 512)
        self.dense2 = nn.Linear(512, 256)

        # Task-specific layers
        self.target_classifier = nn.Linear(256, num_classes)
        self.severity_classifier = nn.Linear(256, num_classes)

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.last_hidden_state[:, 0, :]  # CLS token

        x = self.dropout(pooled_output)
        x = F.relu(self.dense1(x))
        x = self.dropout(x)
        x = F.relu(self.dense2(x))
        x = self.dropout(x)

        # Apply classifiers
        target_output = self.target_classifier(x)
        severity_output = self.severity_classifier(x)

        return target_output, severity_output


# Updated Training Function
def train_model(model, train_loader, val_loader, device, num_epochs=30, learning_rate=2e-5, patience=5):
    optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=1e-4)  # Added weight decay
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=2)
    criterion = nn.CrossEntropyLoss()
    best_val_loss = float('inf')
    early_stop_counter = 0

    for epoch in range(num_epochs):
        model.train()
        total_loss = 0

        for batch in tqdm(train_loader, desc=f"Training Epoch {epoch+1}/{num_epochs}", leave=False):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            target_labels = batch['target_label'].to(device)
            severity_labels = batch['severity_label'].to(device)

            optimizer.zero_grad()
            target_output, severity_output = model(input_ids, attention_mask)

            # Compute losses
            target_loss = criterion(target_output, target_labels)
            severity_loss = criterion(severity_output, severity_labels)
            total_batch_loss = target_loss + severity_loss

            total_batch_loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()
            total_loss += total_batch_loss.item()

        # Validation
        model.eval()
        val_loss = 0
        all_target_preds, all_severity_preds = [], []
        all_target_labels, all_severity_labels = [], []

        with torch.no_grad():
            for batch in tqdm(val_loader, desc="Validating", leave=False):
                input_ids = batch['input_ids'].to(device)
                attention_mask = batch['attention_mask'].to(device)
                target_labels = batch['target_label'].to(device)
                severity_labels = batch['severity_label'].to(device)

                target_output, severity_output = model(input_ids, attention_mask)

                target_loss = criterion(target_output, target_labels)
                severity_loss = criterion(severity_output, severity_labels)
                val_loss += (target_loss + severity_loss).item()

                # Convert logits to predictions
                all_target_preds.extend(torch.argmax(target_output, dim=1).cpu().numpy())
                all_severity_preds.extend(torch.argmax(severity_output, dim=1).cpu().numpy())
                all_target_labels.extend(target_labels.cpu().numpy())
                all_severity_labels.extend(severity_labels.cpu().numpy())

        # Print epoch results
        print(f'Epoch {epoch+1}/{num_epochs}')
        print(f'Training Loss: {total_loss/len(train_loader):.4f}')
        print(f'Validation Loss: {val_loss/len(val_loader):.4f}')
        scheduler.step(val_loss / len(val_loader))

        print("Classification Report for Target Task:")
        print(classification_report(all_target_labels, all_target_preds))
        print("Classification Report for Severity Task:")
        print(classification_report(all_severity_labels, all_severity_preds))

        # # Early stopping
        # if val_loss < best_val_loss:
        #     best_val_loss = val_loss
        #     early_stop_counter = 0
        #     torch.save(model.state_dict(), 'best_model.pth')  # Save the best model
        # else:
        #     early_stop_counter += 1
        #     if early_stop_counter >= patience:
        #         print("Early stopping triggered")
        #         break

# Usage example
def main():
    # Load and preprocess your data
    df_train = pd.read_csv(r'/content/drive/MyDrive/B_trans.csv')  # Training data
    df_val = pd.read_csv(r'/content/drive/MyDrive/B_val_trans.csv')  # Validation data

    # Encode the labels
    df_train, le_target, le_severity = encode_labels(df_train, 'Target', 'Severity')
    df_val, _, _ = encode_labels(df_val, 'Target', 'Severity')  # Apply same encoding

    # Clean and preprocess the data
    df_train.dropna(subset=['Tweet'], inplace=True)
    df_val.dropna(subset=['Tweet'], inplace=True)

    # Extract texts and labels for training and validation
    texts_train = df_train['Tweet'].values
    target_labels_train = df_train['Target'].values
    severity_labels_train = df_train['Severity'].values

    texts_val = df_val['Tweet'].values
    target_labels_val = df_val['Target'].values
    severity_labels_val = df_val['Severity'].values

    # Initialize tokenizer and model
    tokenizer = BertTokenizer.from_pretrained("bert-base-multilingual-uncased")
    bert_model = BertModel.from_pretrained("bert-base-multilingual-uncased")

    # Create datasets
    train_dataset = TargetSeverityDataset(texts_train, target_labels_train, severity_labels_train, tokenizer)
    val_dataset = TargetSeverityDataset(texts_val, target_labels_val, severity_labels_val, tokenizer)

    # Create DataLoaders
    batch_size = 32
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size)

    # Initialize model
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = MultiTaskTargetSeverityClassifier(bert_model).to(device)

    # Train model
    train_model(model, train_loader, val_loader, device)

# Run the main function
if __name__ == '__main__':
    main()


KeyboardInterrupt: 

TRAIN SAVE MODEL

In [ ]:
#12pm 19th nov

import numpy as np
import pandas as pd
import torch
from torch import nn
from transformers import BertTokenizer, BertModel
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import classification_report
import torch.nn.functional as F
from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder

# Function to encode labels
def encode_labels(df, target_column, severity_column):
    le_target = LabelEncoder()
    le_severity = LabelEncoder()

    # Encode 'Target' and 'Severity' labels into integers
    df[target_column] = le_target.fit_transform(df[target_column])
    df[severity_column] = le_severity.fit_transform(df[severity_column])

    return df, le_target, le_severity

# Custom Dataset
class TargetSeverityDataset(Dataset):
    def __init__(self, texts, target_labels, severity_labels, tokenizer, max_len=128):
        self.texts = texts
        self.target_labels = target_labels
        self.severity_labels = severity_labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])
        target_label = self.target_labels[idx]
        severity_label = self.severity_labels[idx]

        encoding = self.tokenizer(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )

        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'target_label': torch.tensor(target_label, dtype=torch.long),
            'severity_label': torch.tensor(severity_label, dtype=torch.long)
        }

# Updated Multi-task Model
class MultiTaskTargetSeverityClassifier(nn.Module):
    def __init__(self, bert_model, dropout_rate=0.4, num_classes=4):
        super().__init__()
        self.bert = bert_model
        hidden_size = bert_model.config.hidden_size

        # Common layers
        self.dropout = nn.Dropout(dropout_rate)
        self.dense1 = nn.Linear(hidden_size, 512)
        self.dense2 = nn.Linear(512, 256)

        # Task-specific layers
        self.target_classifier = nn.Linear(256, num_classes)
        self.severity_classifier = nn.Linear(256, num_classes)

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.last_hidden_state[:, 0, :]  # CLS token

        x = self.dropout(pooled_output)
        x = F.relu(self.dense1(x))
        x = self.dropout(x)
        x = F.relu(self.dense2(x))
        x = self.dropout(x)

        # Apply classifiers
        target_output = F.softmax(self.target_classifier(x), dim=1)
        severity_output = F.softmax(self.severity_classifier(x), dim=1)

        return target_output, severity_output

# Updated Training Function
def train_model(model, train_loader, val_loader, device, num_epochs=30, learning_rate=2e-5, patience=5):
    optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=1e-4)  # Added weight decay
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=2)
    criterion = nn.CrossEntropyLoss()
    best_val_loss = float('inf')
    early_stop_counter = 0

    for epoch in range(num_epochs):
        model.train()
        total_loss = 0

        for batch in tqdm(train_loader, desc=f"Training Epoch {epoch+1}/{num_epochs}", leave=False):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            target_labels = batch['target_label'].to(device)
            severity_labels = batch['severity_label'].to(device)

            optimizer.zero_grad()
            target_output, severity_output = model(input_ids, attention_mask)

            # Compute losses
            target_loss = criterion(target_output, target_labels)
            severity_loss = criterion(severity_output, severity_labels)
            total_batch_loss = target_loss + severity_loss

            total_batch_loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()
            total_loss += total_batch_loss.item()

        # Validation
        model.eval()
        val_loss = 0
        all_target_preds, all_severity_preds = [], []
        all_target_labels, all_severity_labels = [], []

        with torch.no_grad():
            for batch in tqdm(val_loader, desc="Validating", leave=False):
                input_ids = batch['input_ids'].to(device)
                attention_mask = batch['attention_mask'].to(device)
                target_labels = batch['target_label'].to(device)
                severity_labels = batch['severity_label'].to(device)

                target_output, severity_output = model(input_ids, attention_mask)

                target_loss = criterion(target_output, target_labels)
                severity_loss = criterion(severity_output, severity_labels)
                val_loss += (target_loss + severity_loss).item()

                # Convert logits to predictions
                all_target_preds.extend(torch.argmax(target_output, dim=1).cpu().numpy())
                all_severity_preds.extend(torch.argmax(severity_output, dim=1).cpu().numpy())
                all_target_labels.extend(target_labels.cpu().numpy())
                all_severity_labels.extend(severity_labels.cpu().numpy())

        # Calculate accuracy
        target_accuracy = np.mean(np.array(all_target_preds) == np.array(all_target_labels))
        severity_accuracy = np.mean(np.array(all_severity_preds) == np.array(all_severity_labels))

        # Print epoch results
        print(f'\nEpoch {epoch+1}/{num_epochs}')
        print(f'Training Loss: {total_loss/len(train_loader):.4f}')
        print(f'Validation Loss: {val_loss/len(val_loader):.4f}')
        print(f'Target Accuracy: {target_accuracy * 100:.2f}%')
        print(f'Severity Accuracy: {severity_accuracy * 100:.2f}%')
        scheduler.step(val_loss / len(val_loader))

        print("\nClassification Report for Target Task:")
        print(classification_report(all_target_labels, all_target_preds))
        print("\nClassification Report for Severity Task:")
        print(classification_report(all_severity_labels, all_severity_preds))

        # Save model if accuracy thresholds are met
        if target_accuracy >= 0.7 and severity_accuracy >= 0.6:
            torch.save(model.state_dict(), f'/content/drive/MyDrive/taskB.pth')
            print(f"Model saved at epoch {epoch+1} with Target Accuracy: {target_accuracy*100:.2f}% and Severity Accuracy: {severity_accuracy*100:.2f}%")

# Usage example
def main():
    # Load and preprocess your data
    df_train = pd.read_csv(r'/content/drive/MyDrive/B_trans.csv')  # Training data
    df_val = pd.read_csv(r'/content/drive/MyDrive/B_val_trans.csv')  # Validation data

    # Encode the labels
    df_train, le_target, le_severity = encode_labels(df_train, 'Target', 'Severity')
    df_val, _, _ = encode_labels(df_val, 'Target', 'Severity')  # Apply same encoding

    # Clean and preprocess the data
    df_train.dropna(subset=['Tweet'], inplace=True)
    df_val.dropna(subset=['Tweet'], inplace=True)

    # Extract texts and labels for training and validation
    texts_train = df_train['Tweet'].values
    target_labels_train = df_train['Target'].values
    severity_labels_train = df_train['Severity'].values

    texts_val = df_val['Tweet'].values
    target_labels_val = df_val['Target'].values
    severity_labels_val = df_val['Severity'].values

    # Initialize tokenizer and model
    tokenizer = BertTokenizer.from_pretrained("bert-base-multilingual-uncased")
    bert_model = BertModel.from_pretrained("bert-base-multilingual-uncased")

    # Create datasets
    train_dataset = TargetSeverityDataset(texts_train, target_labels_train, severity_labels_train, tokenizer)
    val_dataset = TargetSeverityDataset(texts_val, target_labels_val, severity_labels_val, tokenizer)

    # Create DataLoaders
    batch_size = 32
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size)

    # Initialize model
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = MultiTaskTargetSeverityClassifier(bert_model).to(device)

    # Train model
    train_model(model, train_loader, val_loader, device)

# Run the main function
if __name__ == '__main__':
    main()


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/872k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.72M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

...


Epoch 3/30
Training Loss: 1.9758
Validation Loss: 1.9289
Target Accuracy: 66.33%
Severity Accuracy: 57.07%

Classification Report for Target Task:
              precision    recall  f1-score   support

           0       0.58      0.33      0.42       140
           1       0.85      0.70      0.77       274
           2       0.67      0.61      0.63        99
           3       0.57      0.81      0.67       286

    accuracy                           0.66       799
   macro avg       0.67      0.61      0.62       799
weighted avg       0.68      0.66      0.66       799


Classification Report for Severity Task:
              precision    recall  f1-score   support

           0       0.48      0.53      0.50        74
           1       0.66      0.27      0.38       257
           2       0.54      0.63      0.58       182
           3       0.58      0.81      0.68       286

    accuracy                           0.57       799
   macro avg       0.57      0.56      0.54      


Epoch 4/30
Training Loss: 1.8777
Validation Loss: 1.8889
Target Accuracy: 68.96%
Severity Accuracy: 59.95%

Classification Report for Target Task:
              precision    recall  f1-score   support

           0       0.56      0.39      0.46       140
           1       0.85      0.72      0.78       274
           2       0.69      0.62      0.65        99
           3       0.62      0.83      0.71       286

    accuracy                           0.69       799
   macro avg       0.68      0.64      0.65       799
weighted avg       0.70      0.69      0.68       799


Classification Report for Severity Task:
              precision    recall  f1-score   support

           0       0.53      0.45      0.49        74
           1       0.63      0.37      0.46       257
           2       0.53      0.68      0.60       182
           3       0.64      0.80      0.71       286

    accuracy                           0.60       799
   macro avg       0.58      0.57      0.56      


Epoch 5/30
Training Loss: 1.7682
Validation Loss: 1.8659
Target Accuracy: 69.21%
Severity Accuracy: 60.08%

Classification Report for Target Task:
              precision    recall  f1-score   support

           0       0.58      0.44      0.50       140
           1       0.82      0.75      0.79       274
           2       0.70      0.60      0.64        99
           3       0.63      0.79      0.70       286

    accuracy                           0.69       799
   macro avg       0.68      0.65      0.66       799
weighted avg       0.70      0.69      0.69       799


Classification Report for Severity Task:
              precision    recall  f1-score   support

           0       0.55      0.49      0.52        74
           1       0.63      0.35      0.45       257
           2       0.53      0.70      0.60       182
           3       0.64      0.79      0.71       286

    accuracy                           0.60       799
   macro avg       0.59      0.58      0.57      


Epoch 6/30
Training Loss: 1.6321
Validation Loss: 1.8319
Target Accuracy: 70.21%
Severity Accuracy: 60.83%

Classification Report for Target Task:
              precision    recall  f1-score   support

           0       0.54      0.41      0.47       140
           1       0.79      0.80      0.80       274
           2       0.65      0.71      0.68        99
           3       0.70      0.74      0.72       286

    accuracy                           0.70       799
   macro avg       0.67      0.67      0.67       799
weighted avg       0.70      0.70      0.70       799


Classification Report for Severity Task:
              precision    recall  f1-score   support

           0       0.48      0.57      0.52        74
           1       0.60      0.40      0.48       257
           2       0.53      0.70      0.61       182
           3       0.71      0.74      0.72       286

    accuracy                           0.61       799
   macro avg       0.58      0.60      0.58      

KeyboardInterrupt: 

load the model and generate predictions

In [ ]:
import pandas as pd
import torch
from transformers import BertTokenizer, BertModel
from torch.utils.data import Dataset, DataLoader
from torch import nn
import torch.nn.functional as F
from tqdm import tqdm  # For progress bars
from sklearn.preprocessing import LabelEncoder

# Custom Dataset for Test Data
class TargetSeverityDatasetTest(Dataset):
    def __init__(self, texts, tokenizer, max_len=128):
        self.texts = texts
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])

        encoding = self.tokenizer(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )

        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten()
        }

# Multi-task Model Definition
class MultiTaskTargetSeverityClassifier(nn.Module):
    def __init__(self, bert_model, dropout_rate=0.3, num_target_classes=4, num_severity_classes=4):
        super().__init__()
        self.bert = bert_model
        hidden_size = bert_model.config.hidden_size

        # Common layers
        self.dropout = nn.Dropout(dropout_rate)
        self.dense1 = nn.Linear(hidden_size, 512)
        self.dense2 = nn.Linear(512, 256)

        # Task-specific layers
        self.target_classifier = nn.Linear(256, num_target_classes)
        self.severity_classifier = nn.Linear(256, num_severity_classes)

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.last_hidden_state[:, 0, :]  # CLS token

        x = self.dropout(pooled_output)
        x = F.relu(self.dense1(x))
        x = self.dropout(x)
        x = F.relu(self.dense2(x))
        x = self.dropout(x)

        target_output = F.softmax(self.target_classifier(x), dim=1)
        severity_output = F.softmax(self.severity_classifier(x), dim=1)

        return target_output, severity_output

# Load the saved model
def load_model(model_path, device):
    tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-uncased')  # Use BERT tokenizer
    bert_model = BertModel.from_pretrained('bert-base-multilingual-uncased')  # Use BERT model
    model = MultiTaskTargetSeverityClassifier(bert_model).to(device)
    model.load_state_dict(torch.load(model_path))  # Load trained model weights
    model.eval()
    return model, tokenizer

# Prediction function
def make_predictions(model, tokenizer, test_loader, device):
    all_predictions = []
    with torch.no_grad():
        for batch in tqdm(test_loader, desc="Making Predictions", leave=False, ncols=100, dynamic_ncols=True):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)

            target_output, severity_output = model(input_ids, attention_mask)

            # Get predicted classes (indices with max probability)
            target_preds = torch.argmax(target_output, dim=1).cpu().numpy()
            severity_preds = torch.argmax(severity_output, dim=1).cpu().numpy()

            all_predictions.extend(zip(target_preds, severity_preds))
    return all_predictions

# Main function for making predictions and saving to CSV
def main():
    # Load the test data
    df_test = pd.read_csv(r'/content/drive/MyDrive/Test_trans.csv')  # Replace with your test data path

    # Initialize tokenizer and device
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    # Load the model
    model, tokenizer = load_model(r'/content/drive/MyDrive/taskB.pth', device)  # Replace with your model path

    # Create the test dataset and DataLoader
    test_dataset = TargetSeverityDatasetTest(df_test['Tweet'].values, tokenizer)
    test_loader = DataLoader(test_dataset, batch_size=32)

    # Make predictions
    predictions = make_predictions(model, tokenizer, test_loader, device)

    # Initialize LabelEncoders and mappings
    target_mapping = ['I', 'O', 'R', 'N/A']
    severity_mapping = ['H', 'L', 'M', 'N/A']

    # Map predictions back to original labels
    df_test['Predicted_Target'] = [target_mapping[pred[0]] for pred in predictions]
    df_test['Predicted_Severity'] = [severity_mapping[pred[1]] for pred in predictions]

    # Save predictions to CSV
    df_test[['Id', 'Predicted_Target', 'Predicted_Severity']].to_csv(r'/content/drive/MyDrive/taskB_predictions.csv', index=False)
    print("Predictions saved to 'taskB_predictions.csv'.")

# Run the main function
if __name__ == '__main__':
    main()


<ipython-input-7-4be5ab749fd0>:73: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path))  # Load trained model weights
                

Predictions saved to 'taskB_predictions.csv'.


checking labels

In [ ]:
#12pm 19th nov

import numpy as np
import pandas as pd
import torch
from torch import nn
from transformers import BertTokenizer, BertModel
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import classification_report
import torch.nn.functional as F
from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder

# Function to encode labels
def encode_labels(df, target_column, severity_column):
    le_target = LabelEncoder()
    le_severity = LabelEncoder()

    # Encode 'Target' and 'Severity' labels into integers
    df[target_column] = le_target.fit_transform(df[target_column])
    df[severity_column] = le_severity.fit_transform(df[severity_column])

    return df, le_target, le_severity

# Custom Dataset
class TargetSeverityDataset(Dataset):
    def __init__(self, texts, target_labels, severity_labels, tokenizer, max_len=128):
        self.texts = texts
        self.target_labels = target_labels
        self.severity_labels = severity_labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])
        target_label = self.target_labels[idx]
        severity_label = self.severity_labels[idx]

        encoding = self.tokenizer(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )

        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'target_label': torch.tensor(target_label, dtype=torch.long),
            'severity_label': torch.tensor(severity_label, dtype=torch.long)
        }

# Multi-task Model with Softmax
# Updated Multi-task Model
class MultiTaskTargetSeverityClassifier(nn.Module):
    def __init__(self, bert_model, dropout_rate=0.4, num_classes=4):
        super().__init__()
        self.bert = bert_model
        hidden_size = bert_model.config.hidden_size

        # Common layers
        self.dropout = nn.Dropout(dropout_rate)
        self.dense1 = nn.Linear(hidden_size, 512)
        self.dense2 = nn.Linear(512, 256)

        # Task-specific layers
        self.target_classifier = nn.Linear(256, num_classes)
        self.severity_classifier = nn.Linear(256, num_classes)

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.last_hidden_state[:, 0, :]  # CLS token

        x = self.dropout(pooled_output)
        x = F.relu(self.dense1(x))
        x = self.dropout(x)
        x = F.relu(self.dense2(x))
        x = self.dropout(x)

        # Apply classifiers
        target_output = self.target_classifier(x)
        severity_output = self.severity_classifier(x)

        return target_output, severity_output


# Updated Training Function
# Updated Training Function
def train_model(model, train_loader, val_loader, device, num_epochs=30, learning_rate=2e-5, patience=5):
    optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=1e-4)  # Added weight decay
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=2)
    criterion = nn.CrossEntropyLoss()
    best_val_loss = float('inf')
    early_stop_counter = 0

    for epoch in range(num_epochs):
        model.train()
        total_loss = 0

        for batch in tqdm(train_loader, desc=f"Training Epoch {epoch+1}/{num_epochs}", leave=False):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            target_labels = batch['target_label'].to(device)
            severity_labels = batch['severity_label'].to(device)

            optimizer.zero_grad()
            target_output, severity_output = model(input_ids, attention_mask)

            # Compute losses
            target_loss = criterion(target_output, target_labels)
            severity_loss = criterion(severity_output, severity_labels)
            total_batch_loss = target_loss + severity_loss

            total_batch_loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()
            total_loss += total_batch_loss.item()

        # Validation
        model.eval()
        val_loss = 0
        all_target_preds, all_severity_preds = [], []
        all_target_labels, all_severity_labels = [], []

        with torch.no_grad():
            for batch in tqdm(val_loader, desc="Validating", leave=False):
                input_ids = batch['input_ids'].to(device)
                attention_mask = batch['attention_mask'].to(device)
                target_labels = batch['target_label'].to(device)
                severity_labels = batch['severity_label'].to(device)

                target_output, severity_output = model(input_ids, attention_mask)

                target_loss = criterion(target_output, target_labels)
                severity_loss = criterion(severity_output, severity_labels)
                val_loss += (target_loss + severity_loss).item()

                # Convert logits to predictions
                all_target_preds.extend(torch.argmax(target_output, dim=1).cpu().numpy())
                all_severity_preds.extend(torch.argmax(severity_output, dim=1).cpu().numpy())
                all_target_labels.extend(target_labels.cpu().numpy())
                all_severity_labels.extend(severity_labels.cpu().numpy())

        # Calculate accuracy
        target_accuracy = np.mean(np.array(all_target_preds) == np.array(all_target_labels))
        severity_accuracy = np.mean(np.array(all_severity_preds) == np.array(all_severity_labels))

        # Print epoch results
        print(f'\nEpoch {epoch+1}/{num_epochs}')
        print(f'Training Loss: {total_loss/len(train_loader):.4f}')
        print(f'Validation Loss: {val_loss/len(val_loader):.4f}')
        print(f'Target Accuracy: {target_accuracy * 100:.2f}%')
        print(f'Severity Accuracy: {severity_accuracy * 100:.2f}%')
        scheduler.step(val_loss / len(val_loader))

        print("\nClassification Report for Target Task:")
        print(classification_report(all_target_labels, all_target_preds))
        print("\nClassification Report for Severity Task:")
        print(classification_report(all_severity_labels, all_severity_preds))

        # Save model if accuracy thresholds are met
        if target_accuracy >= 0.7 and severity_accuracy >= 0.6:
            torch.save(model.state_dict(), f'/content/drive/MyDrive/taskB.pth')
            print(f"Model saved at epoch {epoch+1} with Target Accuracy: {target_accuracy*100:.2f}% and Severity Accuracy: {severity_accuracy*100:.2f}%")

def display_encoded_mappings(label_encoder, column_name):
    print(f"{column_name} Labels (Encoded -> Original):")
    for encoded, original in enumerate(label_encoder.classes_):
        print(f"{encoded} -> {original}")
    print()

# Usage example
def main():
    # Load and preprocess your data
    df_train = pd.read_csv(r'/content/drive/MyDrive/B_trans.csv')  # Training data
    df_val = pd.read_csv(r'/content/drive/MyDrive/B_val_trans.csv')  # Validation data

    # Encode the labels
    df_train, le_target, le_severity = encode_labels(df_train, 'Target', 'Severity')
    df_val, _, _ = encode_labels(df_val, 'Target', 'Severity')  # Apply same encoding

    display_encoded_mappings(le_target, "Target")
    display_encoded_mappings(le_severity, "Severity")
    # Clean and preprocess the data
    df_train.dropna(subset=['Tweet'], inplace=True)
    df_val.dropna(subset=['Tweet'], inplace=True)

    # Extract texts and labels for training and validation
    texts_train = df_train['Tweet'].values
    target_labels_train = df_train['Target'].values
    severity_labels_train = df_train['Severity'].values

    texts_val = df_val['Tweet'].values
    target_labels_val = df_val['Target'].values
    severity_labels_val = df_val['Severity'].values

    # Initialize tokenizer and model
    tokenizer = BertTokenizer.from_pretrained("bert-base-multilingual-uncased")
    bert_model = BertModel.from_pretrained("bert-base-multilingual-uncased")

    # Create datasets
    train_dataset = TargetSeverityDataset(texts_train, target_labels_train, severity_labels_train, tokenizer)
    val_dataset = TargetSeverityDataset(texts_val, target_labels_val, severity_labels_val, tokenizer)

    # Create DataLoaders
    batch_size = 32
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size)

    # Initialize model
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = MultiTaskTargetSeverityClassifier(bert_model).to(device)

    # Train model
    train_model(model, train_loader, val_loader, device)

# Run the main function
if __name__ == '__main__':
    main()


Target Labels (Encoded -> Original):
0 -> I
1 -> O
2 -> R
3 -> nan

Severity Labels (Encoded -> Original):
0 -> H
1 -> L
2 -> M
3 -> nan



KeyboardInterrupt: 

checking the accuracy of predicted labels

In [ ]:
#12pm 19th nov

import numpy as np
import pandas as pd
import torch
from torch import nn
from transformers import BertTokenizer, BertModel
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import classification_report
import torch.nn.functional as F
from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder

# Function to encode labels
def encode_labels(df, target_column, severity_column):
    le_target = LabelEncoder()
    le_severity = LabelEncoder()

    # Encode 'Target' and 'Severity' labels into integers
    df[target_column] = le_target.fit_transform(df[target_column])
    df[severity_column] = le_severity.fit_transform(df[severity_column])

    return df, le_target, le_severity

# Custom Dataset
class TargetSeverityDataset(Dataset):
    def __init__(self, texts, target_labels, severity_labels, tokenizer, max_len=128):
        self.texts = texts
        self.target_labels = target_labels
        self.severity_labels = severity_labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])
        target_label = self.target_labels[idx]
        severity_label = self.severity_labels[idx]

        encoding = self.tokenizer(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )

        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'target_label': torch.tensor(target_label, dtype=torch.long),
            'severity_label': torch.tensor(severity_label, dtype=torch.long)
        }

# Multi-task Model with Softmax
# Updated Multi-task Model
class MultiTaskTargetSeverityClassifier(nn.Module):
    def __init__(self, bert_model, dropout_rate=0.4, num_classes=4):
        super().__init__()
        self.bert = bert_model
        hidden_size = bert_model.config.hidden_size

        # Common layers
        self.dropout = nn.Dropout(dropout_rate)
        self.dense1 = nn.Linear(hidden_size, 512)
        self.dense2 = nn.Linear(512, 256)

        # Task-specific layers
        self.target_classifier = nn.Linear(256, num_classes)
        self.severity_classifier = nn.Linear(256, num_classes)

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.last_hidden_state[:, 0, :]  # CLS token

        x = self.dropout(pooled_output)
        x = F.relu(self.dense1(x))
        x = self.dropout(x)
        x = F.relu(self.dense2(x))
        x = self.dropout(x)

        # Apply classifiers
        target_output = self.target_classifier(x)
        severity_output = self.severity_classifier(x)

        return target_output, severity_output


# Updated Training Function
def train_model(model, train_loader, val_loader, device, num_epochs=30, learning_rate=2e-5, patience=5):
    optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=1e-4)  # Added weight decay
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=2)
    criterion = nn.CrossEntropyLoss()
    best_val_loss = float('inf')
    early_stop_counter = 0

    for epoch in range(num_epochs):
        model.train()
        total_loss = 0

        for batch in tqdm(train_loader, desc=f"Training Epoch {epoch+1}/{num_epochs}", leave=False):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            target_labels = batch['target_label'].to(device)
            severity_labels = batch['severity_label'].to(device)

            optimizer.zero_grad()
            target_output, severity_output = model(input_ids, attention_mask)

            # Compute losses
            target_loss = criterion(target_output, target_labels)
            severity_loss = criterion(severity_output, severity_labels)
            total_batch_loss = target_loss + severity_loss

            total_batch_loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()
            total_loss += total_batch_loss.item()

        # Validation
        model.eval()
        val_loss = 0
        all_target_preds, all_severity_preds = [], []
        all_target_labels, all_severity_labels = [], []

        with torch.no_grad():
            for batch in tqdm(val_loader, desc="Validating", leave=False):
                input_ids = batch['input_ids'].to(device)
                attention_mask = batch['attention_mask'].to(device)
                target_labels = batch['target_label'].to(device)
                severity_labels = batch['severity_label'].to(device)

                target_output, severity_output = model(input_ids, attention_mask)

                target_loss = criterion(target_output, target_labels)
                severity_loss = criterion(severity_output, severity_labels)
                val_loss += (target_loss + severity_loss).item()

                # Convert logits to predictions
                all_target_preds.extend(torch.argmax(target_output, dim=1).cpu().numpy())
                all_severity_preds.extend(torch.argmax(severity_output, dim=1).cpu().numpy())
                all_target_labels.extend(target_labels.cpu().numpy())
                all_severity_labels.extend(severity_labels.cpu().numpy())

        # Print epoch results
        print(f'Epoch {epoch+1}/{num_epochs}')
        print(f'Training Loss: {total_loss/len(train_loader):.4f}')
        print(f'Validation Loss: {val_loss/len(val_loader):.4f}')
        scheduler.step(val_loss / len(val_loader))

        print("Classification Report for Target Task:")
        print(classification_report(all_target_labels, all_target_preds))
        print("Classification Report for Severity Task:")
        print(classification_report(all_severity_labels, all_severity_preds))

        # # Early stopping
        # if val_loss < best_val_loss:
        #     best_val_loss = val_loss
        #     early_stop_counter = 0
        #     torch.save(model.state_dict(), 'best_model.pth')  # Save the best model
        # else:
        #     early_stop_counter += 1
        #     if early_stop_counter >= patience:
        #         print("Early stopping triggered")
        #         break

# Usage example
def main():
    # Load and preprocess your data
    df_train = pd.read_csv(r'/content/drive/MyDrive/B_trans.csv')  # Training data
    df_val = pd.read_csv(r'/content/drive/MyDrive/taskB_predictions.csv')  # Validation data

    # Encode the labels
    df_train, le_target, le_severity = encode_labels(df_train, 'Target', 'Severity')
    df_val, _, _ = encode_labels(df_val, 'Target', 'Severity')  # Apply same encoding

    # Clean and preprocess the data
    df_train.dropna(subset=['Tweet'], inplace=True)
    df_val.dropna(subset=['Tweet'], inplace=True)

    # Extract texts and labels for training and validation
    texts_train = df_train['Tweet'].values
    target_labels_train = df_train['Target'].values
    severity_labels_train = df_train['Severity'].values

    texts_val = df_val['Tweet'].values
    target_labels_val = df_val['Target'].values
    severity_labels_val = df_val['Severity'].values

    # Initialize tokenizer and model
    tokenizer = BertTokenizer.from_pretrained("bert-base-multilingual-uncased")
    bert_model = BertModel.from_pretrained("bert-base-multilingual-uncased")

    # Create datasets
    train_dataset = TargetSeverityDataset(texts_train, target_labels_train, severity_labels_train, tokenizer)
    val_dataset = TargetSeverityDataset(texts_val, target_labels_val, severity_labels_val, tokenizer)

    # Create DataLoaders
    batch_size = 32
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size)

    # Initialize model
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = MultiTaskTargetSeverityClassifier(bert_model).to(device)

    # Train model
    train_model(model, train_loader, val_loader, device)

# Run the main function
if __name__ == '__main__':
    main()
